# Inlämningsuppgift 2: Utgående långvågig strålning (OLR)

Den här uppgiften genomförs och lämnas in individuellt, men ni får gärna diskutera och hjälpa varandra.
Du antas ha gått igenom [Python-introduktionen](01_introduktion.ipynb) innan du börjar på uppgiften.

Inlämningen sker via Canvas, under Startsida $\rightarrow$ Inlämningar $\rightarrow$ Inlämning 2: Utgående långvågig strålning.

## Mål

I denna uppgift ska du simulera utgående långvågig strålning och hur den påverkas av förändringar i koncentrationen av några gaser och temperaturprofilen.
Tanken med uppgiften är att illustrera mer exakt varför stigande halter av så kallade växthusgaser är problematisk, d.v.s. i detalj visa hur gaserna påverkar jordens kylmekanism.
På engelska kallas den utgående strålningen OLR (outgoing longwave radiation) och den termen används även här för enkelhets skull.

Uppgiften är utformad för att träna er i att utföra beräkningar i Python.
Speciellt ger uppgiften träning i att numeriskt integrera funktioner. Detta gör du enklast med NumPy-funktionen `trapz`.
Läs dokumentationen för `trapz` så du vet hur den fungerar.
Uppgiften ger också träning i att hantera data som har flera dimensioner och hur sådant data kan representeras som tvådimensionella fält (matriser, eller mer generellt i NumPy, arrays).
Det senare är en tillämpning av det ni har lärt er i Linjär algebra.

## Avgränsningar

Simuleringarna är ytterst detaljerade när det gäller gasernas absorptionsspektra.
Men för att göra problemet hanterbart så bortser vi ifrån moln.
Vi antar även att marken agerar som en svart kropp och att atmosfären inte har någon horisontell variation (båda är goda approximationer).
Datan som du får täcker höjder upp till 20 km och representerar tropiska förhållande med förindustriella gasmängder.

## Teori

Fysikaliskt sett är OLR jordens emittans, $E$ (enhet: Wm<sup>-2</sup>).
Det går ej att beräkna emittansen direkt, utan den fås genom att integrera den spektrala emittansen, $E_s$ (enhet: Wm<sup>-2</sup>Hz<sup>-1</sup>), över alla frekvenser ($\nu$): 
$$
E = \int_{v=0}^\infty E_s(\nu)d\nu,
\tag{1}
$$

Även $E_s$ är ett integrerat värde.
Den fås genom att integrera (som att summera, men för en kontinuerlig variabel) all utgående strålning.
Den tekniska termen för strålning som vi använder här är spektral radians,
$I$ (enhet: Wm<sup>-2</sup>Hz<sup>-1</sup>sr<sup>-1</sup>, där sr står för steradian),
som beror på zenitvinkeln $\theta$ och azimutvinkeln $\phi$,
se Figur 1 nedan.
T.ex. ger en zenitvinkel på 0° kortast väg genom atmosfären,
medan en zenitvinkel nära 90° ger en relativt lång väg.

<figure>
   <img src="./media/zenith-azimuth-schematic.svg" width=100%>
    <figcaption>
        <b>Figur 1: </b>(a) Illustration av utgående spektral radians för en viss vinkel.
        (b) Mer detaljerad bild av zenit- och azimutvinkel för utgående spektral radians.
    </figcaption>
</figure>

Vi vill alltså integrera ("summera") all spektral radians som "går uppåt" ut från atmosfären,
d.v.s. över en halvsfär vid toppen av atmosfären:

$$
E_s(\nu) = \int_{\phi=0}^{2\pi} \int_{\theta=0}^{\pi / 2} I(\nu, \theta, \phi)\cos(\theta)\sin(\theta)d\theta d\phi
\tag{2} 
$$
Ekvation (2) är en dubbelintegral, men låt inte det skrämma dig!
För de förhållanden som beskrivs under [Avgränsningar](#Avgränsningar) så beror inte $I$ på azimuthvinkeln $\phi$, och uttrycket kan då förenklas som

$$
E_s(\nu) = \int_{\phi=0}^{2\pi} d\phi \cdot \int_{\theta=0} ^{\pi / 2} I(\nu, \theta) \cos(\theta) \sin(\theta) d\theta,
\tag{3}
$$

alltså en multiplikation av två integraler.
Den första integralen kan lösas analytiskt och är bara ett konstant värde (låt oss kalla det $C$ för enkelhets skull).
Så börja med att lösa följande integral analytiskt:
$$
C = \int_{\phi = 0}^{2\pi} d\phi = ?
\tag{4}
$$

Uppgiften är nu alltså att räkna ut
$$
E_s(\nu) = C \cdot \int_{\theta=0}^{\pi/2} I(\nu, \theta)\cos(\theta)\sin(\theta) d\theta.
\tag{5}
$$

Vi går inte in på hur $I(\nu, \theta)$ beräknas, utan du får en färdig funktion som räknar ut $I$ för alla frekvenser för ett angivet värde på $\theta$.
Integralen i Ekvation (5) löser vi numeriskt med NumPy-funktionen `trapz`.

Nedan kommer du få i uppgift att beräkna spektrala emittanser $E_s$ och totala emittanser $E$.

### Frekvens och vågtal
I figurer med strålning är det vanligt att frekvensen anges som “vågtal”, med enhet cm<sup>−1</sup>.
Detta vågtal är $1/\lambda$ där $\lambda$ är våglängden i cm.
Denna enhet kallas också för Kayser ([https: //en.wikipedia.org/wiki/Wavenumber](https://en.wikipedia.org/wiki/Wavenumber)).
Som exempel så visas Planckfunktionen som en funktion av vågtal i Figur 2.

<figure>
    <img src="./media/planck.png" width=600>
    <figcaption>
        <b>Figur 2: </b>Planck-kurvorna för två olika temperaturer som funktion av vågtal.
    </figcaption>
</figure>

## Arbetsgång
<div class="alert alert-block alert-info">
<b>Om du inte använt Python innan rekommenderar vi att du arbetar igenom <a href="01_introduktion.ipynb">Introduktion till Python</a> först!</b>
</div>

- Bestäm om du ska fortsätta jobba på MyBinder eller installera Python och JupyterLab på din egen dator.
  - Se [Installera Python på din dator](#Installera-Python-på-din-dator) för mer information.
- Gå igenom [Praktiska instruktioner](#Praktiska-instruktioner).
- Svara på [Frågorna](#Frågor).

<div class="alert alert-block alert-warning">
<b>Varning: </b>I fallet att du följt länken på Canvas körs den här notebooken på något som heter <a href=https://mybinder.org/>MyBinder</a>, vilket innebär att JupyterLab körs på en server någonstans i molnet. Det är väldigt smidigt för att komma igång med Python och Jupyter, men <b>ingenting sparas</b> och programmet stängs av efter en stunds inaktivitet.
Alltså måste du ladda ner filerna till din egen dator för att spara dem (högerklicka på filen du vill spara i filhanteraren -> Download).
Du måste sedan ladda upp filen igen (genom att klicka på knappen som ser ut som en uppåtpil i filhanteraren) när du startar en ny session.
</div>

### Installera Python på din dator

För att minska risken att förlora arbete rekommenderar vi att du installerar Python och JupyterLab på din egen dator.
Om du inte vet hur, rekommenderar vi [Anaconda](https://www.anaconda.com/download).
Se till att följande bibliotek är installerade (följer med som standard i Anaconda, se *Environments*):

- numpy
- matplotlib
- jupyterlab

### Jobba med inlämningsuppgiften på din dator

Efter att ha installerat Anaconda, öppna programmet.
Starta sedan **JupyterLab** från Home.

Om du redan laddat ner filerna när du gjorde Python-introduktionen kan du hoppa till sista steget.
Annars kan du ladda ner inlämningsuppgiften som en zip här:
https://github.com/hanschen/SEE060_OLR/archive/refs/heads/main.zip

Packa upp zipfilen på ett lämpligt ställe på din dator.

Öppna slutligen `08_uppgift_2_olr.ipynb` i JupyterLab via filhanteraren till vänster.

## Kod och data

Funktioner för att beräkna radians finns i modulen `olr.py` i mappen `Kod`.
Datan kommer i två varianter i mappen `Data`,
där filerna heter `olr_data.npz` och `olr_data large.npz`.
Båda dessa filer innehåller följande variabler:

| | | |
|-|-|-|
| **f**    |Frekvenser [Hz]               |En vektor (endimensionell array). | 
| **wn**   |Vågtal [cm<sup>-1</sup>]      |En vektor, samma längd som f. | 
| **z**    |Höjder [m]                    |En vektor. | 
| **p**    |Atmosfäriskt tryck [Pa]       |En vektor, samma längd som z. |
| **t**    |Atmosfäriskt temperatur [K]   |En vektor, samma längd som z. |
| **vmr**  |Volymandelar [-]              |En matris med dimensioner (gas, z). Inkluderade gaser, i ordning, är:<br>H<sub>2</sub>O, CO<sub>2</sub>, O<sub>3</sub>, CH<sub>4</sub> och N<sub>2</sub>O. |
| **xsec**  |Absorptionstvärsnitt [m<sup>2</sup>] |En array med dimensioner (gas, f, z). Denna variabel kommer ni inte<br>använda direkt, den används av funktioner som beräknar radians. |

Använd först och främst `olr_data.npz`, som innehåller data för 3500 frekvenser.

Om du är nyfiken och vill se resultaten i en högre upplösning kan du byta till `olr_data_large.npz` när du är klar med uppgiften.
Den här filen innehåller data för 35000 frekvenser.

Datafilerna laddas in i Python med NumPys funktion `load`.

<div class="alert alert-block alert-success">
    <b>Uppgift: </b>I cellerna nedan, importera NumPy och ladda in filen Data/olr_data.npz.
</div>

In [ ]:
# Importera numpy

In [ ]:
# Ladda in data till en variabel med namnet data

Med följande syntax tilldelar vi datan för frekvenserna till variablen `f`:

In [ ]:
f = data["f"]

<div class="alert alert-block alert-success">
    <b>Uppgift: </b>Skapa en variabel för varje fysisk variabel i tabellen ovan. Lägg till så många celler nedan som du behöver.
</div>

Vi kommer att använda funktioner från modulen `olr` som finns i filen `Kod/olr.py`.
Vi importerar modulen `Kod.olr` och namnger den till `olr`:

In [ ]:
import Kod.olr as olr

Med till exempel `help(olr)` eller `help(olr.spectral_radiance)` kan du ta reda på vilka funktioner som finns i modulen och hur de fungerar.

<div class="alert alert-block alert-success">
    <b>Uppgift:</b> Visa dokumentationen för <tt>olr.spectral_radiance</tt> nedan med hjälp av <tt>help</tt>-funktionen.
</div>

## Praktiska instruktioner

Nedan antas det att du har importerat modulen olr som `olr` och NumPy som `np`.

### Steg 1

Ladda in datan (om du inte gjort det redan) och kontrollera storleken på varje variabel.
Försök förstå vad de olika dimensionerna betyder.
T.ex. vilken dimension representerar höjd i `vmr`?

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Steg 2
Bli bekant med funktionen `olr.spectral radiance`.
Den anropas som:
```python
olr.spectral_radiance(f, z, p, t, vmr, xsec, za)
```
där `za` är ett värde för zenit-vinkeln (“zenith angle”) i <u>radianer</u>.

<div class="alert alert-block alert-success">
<b>Uppgift:</b>
    Plotta resultatet av funktionen för några vinklar.
    Glöm inte att importera <tt>matplotlib.pyplot</tt>,
    förslagsvis som <tt>plt</tt>.
    Som alltid, lägg till fler celler om du behöver.
</div>

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

<div class="alert alert-block alert-info">
    <b>Tips:</b>
    Får du <tt>RuntimeWarning</tt>-varningar och en array med <tt>nan</tt> och/eller <tt>inf</tt>?
    Kontrollera att alla parametrar till funktionen är passande och med <u>korrekt enhet</u>.
</div>

### Steg 3

Beräkna spektral emittans $E_s$ för alla frekvenser i datan enligt Ekvation (5),
d.v.s.,
gör uträkningen

$$
E_s(\nu) = C \cdot \int_{\theta=0}^{\pi/2} I(\nu, \theta)\cos(\theta)\sin(\theta) d\theta,
\tag{5}
$$

där $C$ är värdet du fick när du integrerade Ekvation (4) analytiskt,
och $I$ erhålls från funktionen `olr.spectral_radiance`.

Notera att vi vill ha $E_s$ för alla frekvenser ($\nu$) i datafilen.
Normalt sett hade vi behöva iterera ("loopa") över alla $\nu$-värden,
men som tur är returnerar funktionen `olr.spectral_radiance` redan spektral radians för alla angivna frekvenser.
Vi behöver dock fortfarande beräkna uttrycket inne i integralen ([integranden](https://sv.wiktionary.org/wiki/integrand)) för ett antal zenitvinklar ($\theta$),
vilket lättast kan göras med en loop.

<div class="alert alert-block alert-info">
<b>Tips:</b> Börja med några få zenitvinklar så går snabbare att köra koden.
    När du är nöjd med koden kan du öka antalet zenitvinklar för att få en mer korrekt beräkning.
</div>

I det här steget finns det några detaljer som är svåra att uppfatta om man inte är erfaren,
så här kommer lite extra hjälp:

1. Till att börja med vill vi beräkna $I(\nu, \theta)\cos(\theta)\sin(\theta)$ för ett antal värden på $\theta$.
   (Vilka värden?
   Titta på integralen i Ekvation (5) igen.)
3. När vi anropar `olr.spectral_radiance` får vi $I$ för ett specifikt värde av $\theta$ och alla angivna frekvenser $\nu$,
   alltså en endimensionell array,
   ett värde för varje frekvens.
   Dubbelkolla att detta stämmer.
   Det är dock inte $\nu$ vi vill integrera över,
   så det är inte den här endimensionella arrayen vi ska integrera.
5. Snabb sammanfattning:
   Vi beräknar $I\cos(\theta)\sin(\theta)$ för ett antal $\theta$,
   och varje gång får vi en endimensionell array.
6. Nu kan man göra något smart:
   vi kan spara allt resultat i en tvådimensionell matris,
   där resultatet för varje uträkning ($I\cos(\theta)\sin(\theta)$ för ett visst $\theta$)
   sparas som en rad eller kolumn i resultat-matrisen.

<div class="alert alert-block alert-info">
<b>Tips: </b>Kommer du ihåg hur vi skapade en tom tvådimensionell array i Python-introduktionen?
    <details>
        <summary><em>Tips:</em></summary>
        1. Det har med nollor att göra.
        <br>
        2. Det kan även hjälpa att titta på exempelkoden i Problem 4 i <a href="07_programmeringsproblem.ipynb">Programmeringsproblem</a>.
    </details>
</div>

5. Målet är alltså att skapa en tvådimensionell matris med dimensionerna ($\theta$, $\nu$) eller ($\nu$, $\theta$)
   som innehåller alla integrand-värden.
   Du kan nu enkelt integrera över den här matrisen med `np.trapz` längs dimensionen som motsvarar $\theta$.

<div class="alert alert-block alert-info">
<b>Tips: </b>Vet du hur man integrerar över en tvådimensionell matris med <tt>trapz</tt>?
    <details>
        <summary><em>Tips:</em></summary>
        Se Problem 5 i <a href="07_programmeringsproblem.ipynb">Programmeringsproblem</a>.
    </details>
</div>

6. Beräkna slutligen $E_s$ (glöm inte konstanten $C$).

Det är svårt i det här skedet att kontrollera om resultatet stämmer.
Det här är vanligt i programmering/dataanalys och något man får lära sig hantera/leva med.
För att göra det lättare för er får ni dock en liten ledtråd:
om du plottar den spektrala emittansen så borde de större värdena ha en storleksordning på 10<sup>-11</sup> Wm<sup>-2</sup>Hz<sup>-1</sup>.

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Steg 4

Vi kommer upprepa beräkningen av spektral emittans för olika förhållanden,
så implementera en funktion som utför uträkningen i [Steg 3](#Steg-3),
som du kallar `spectral_exitance`.
Det ska vara möjligt utföra beräkningen med funktionen för godtycklig temperaturprofil, vertikal sammansättning av gaserna, osv.
Fundera på vad för parametrar din funktionen behöver som input.

<div class="alert alert-block alert-info">
<b>Tips: </b> Titta på vilka argument <tt>olr.spectral_radiance</tt> tar, och anpassa din funktion därefter.
</div>

Testa funktionen med värdena från datafilen.
Kontrollera att resultatet stämmer med det du fick från föregående steg.

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Steg 5

Beräkna nu den totala emittansen genom att integrera över alla frekvenser enligt Ekvation (1):

$$
E = \int_{v=0}^\infty E_s(\nu)d\nu,
\tag{1}
$$

Notera att det teoretiska intervallet går från 0 till oändligheten,
men här integrerar vi över alla frekvenser som ges i inputdatan.

Resultatet ger dig simulerad OLR för jorden under de förhållanden som angavs i [Avgränsningar](#Avgränsningar).
Wow! 🎉

Det här är något vi kan relatera till.
Reflektera över om värdet du fick är rimligt.

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Step 6

Implementera även en funktion för beräkningen i [Steg 5](#Steg-5) som du kallar `exitance`.
Denna funktion ska bland annat anropa `spectral_exitance` som du skapade i [Steg 4](#Steg-4) för att beräkna den spektrala emittansen.
Likt `spectral_exitance` ska den nya `exitance`-funktionen fungera för godtycklig temperaturprofil, vertikal sammansättning av gaserna, osv.
Vad för input kommer funktionen behöva?

### Steg 7
Nu ska du implementa funktioner för att "störa" temperatur- och vmr-profilen för att se hur det påverkar OLR.

<div class="alert alert-block alert-warning">
<b>Varning:</b>
    Python och NumPy är lite trixiga här eftersom de föredrar att jobba med referenser.
    Vad innebär detta?
    Jo, i många programmeringsspråk kopieras variabler till funktioner,
    och man behöver inte oroa sig för att råka modifiera originalvariabeln när man gör något i funktioner.
    Detta gäller alltså inte för Python.
    För att göra en kopia av en NumPy-array kan man använda <tt>.copy</tt>-metoden,
    t.ex. <tt>copy_of_t = t.copy()</tt>.
</div>

Börja med att definera en funktion `perturb_t` för att störa temperaturprofilen:
```python
perturb_t(t, dt)
```
där `t` är orginalprofilen och `dt` är en störning i K.

Vi vill begränsa störningen till höjder inom troposfären.
Vi kan här anta att `t` är ordnad från lägre till högre höjder och att tropopausen är där temperaturprofilen har sitt minsta värde.
Funktionen ska alltså ta en temperaturprofil `t` och ett värde `dt` och lägga till `dt` till `t` överallt i troposfären: alltså från början av arrayen t.o.m. där `t` har sitt minsta värde, men inte till elementen därefter.

<div class="alert alert-block alert-info">
<details>
    <summary><em>Om du behöver tips:</em></summary>
    1. Om du behöver färska upp minnet om hur indexering och slices av numpy arrays fungerar, gå tillbaka till introduktionen.
    <br>
    2. Undersök i dokumentationen eller via din favoritsökmotor om NumPy har en funktion för att hitta indexet för en arrays lägsta värde.
    <br>
    3. Det kan ofta underlätta att t.ex. plotta <tt>t</tt> för att visuellt se om du har gjort rätt.
    <br>
    4. Fortfarande fast? Det kan hjälpa att börja med Problem 6 i <a href="07_programmeringsproblem.ipynb">Programmeringsproblem</a>.
</details>
</div>

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

Skapa en liknande funktion för volymandelar:
```python
perturb_vmr(vmr, igas, dvmr, t)
```
där `vmr` är volymandelar,
`igas` anger indexet för gasen som ska störas,
och `dvmr` är en relativ störning (0.1 för 10% ökning o.s.v.).
Även `perturb_vmr` ska bara störa värdena i troposfären (och `t` måste därför vara input även till den här funktionen).

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

## Frågor
Svaren till följande frågor/deluppgifter ska lämnas in individuellt på Canvas.

<div class="alert  alert-block alert-info">
Kommer du ihåg hur man sparar figurer? Om inte, gå tillbaka till Introduktion till Python och avsnittet om Matplotlib.
</div>

### Fråga 1
Plotta (i samma figur) spektral radians som funktion av vågtal för två zenitvinklar: 10° och 80°. Det ska framgå tydligt i figuren vilken som kurva som motsvarar vilken vinkel. För denna figur, och alla kommande, ange storhet och enhet för bägge axlarna.

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 2
Plotta spektral emittans (glöm ej enhet) som funktion av vågtal.

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 3
Beräkna OLR (emittans) för nominell data,
d.v.s. för värden som kommer ifrån datafilen.
Vilket värde får du?
Enhet?
Hur kom du fram till att ditt värde är rimligt?

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 4
Gör en figur som visar hur spektral emittans ändras för en ökning av temperaturen i troposfären med 1 K.
Vad är förändringen av OLR? (Tänk på tecknet.)

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 5
Gör en figur som visar hur spektral emittans ändras för en 35%-ig ökning av CO<sub>2</sub> i troposfären.
Vad är förändringen av OLR?
35% är ungefär så mycket CO<sub>2</sub> hade ökat ifrån sitt förindustriella värde 2019.
Hur väl stämmer ditt resultat med IPCCs värde?
(Se Figur 3 nedan.)
<figure>
    <img src="./media/ipcc_5_ar6.png" width="800">
    <figcaption>
        <b>Figur 3: </b>Uppskattning av "radiative forcing". Från IPCC AR6, 2021: <em>Climate Change 2021: The Physical Science Basis</em>.
    </figcaption>
</figure>

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 6

Som 5 men modifiera O<sub>3</sub> och jämför med IPCC.
Ökningen av O<sub>3</sub> varierar kraftigt mellan regioner, men den globala ökningen av troposfäriskt ozon är i storleksordningen 30%.

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 7

Som 5 men modifiera CH<sub>4</sub> med ett procenttal som representerar ökningen 2019, och jämför med IPCC.

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 8

Som 5 men modifiera N<sub>2</sub>O med ett procental som representerar ökningen 2019, och jämför med IPCC.

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 9

Ungefär hur mycket måste du ändra H<sub>2</sub>O för att få samma ändring i OLR som den 35%-iga ökningen i CO<sub>2</sub> gav?
Om du slår samman resultaten ifrån 5–9, vilken är den starkaste växthusgasen när det kommer till relativa förändringar?

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 10

Hur mycket förändras OLR vid en fördubbling av CO<sub>2</sub>?
Vilken temperaturökning behövs för att föra OLR tillbaka (d.v.s. till OLR-värdet ifrån fråga 3)?

(Tips:
Du kan hitta temperaturvärdet med "trial and error".
Men kan du även beräkna den med någon fysikalisk lag?)

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 11

**Frivillig del**

Kan du använda resultatet från uppgift 10 för att uppskatta storleken på vattenångans återkoppling som respons på en fördubbling av CO<sub>2</sub>?
Det vill säga, hur stor blir temperaturökningen om man inkluderar återkoppling av vattenånga?

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

### Fråga 12

**Frivillig del**

Kan du verifiera att inverkan av CO<sub>2</sub> är ungefärligt logaritmisk?
Det vill säga att inverkan följer $k \cdot \ln(C/C_0 )$, där $C_0$ är en referenskoncentration.
Enligt [https://en.wikipedia.org/wiki/Radiative_forcing#Forcing_due_to_changes_in_atmospheric_gas](https://en.wikipedia.org/wiki/Radiative_forcing#Forcing_due_to_changes_in_atmospheric_gas) är $k = 5.35$ W/m$^2$.
Vilket värde får du?

In [ ]:
# Lägg till din kod här, lägg till fler celler om det behövs

## Avslutning

Ladda upp dina svar på uppgifterna ovan på Canvas.